In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import dbcp

# EIA 860m vs Interconnection Queues

Key questions:
* EIA data is reported at the generator level, while interconnection queues are reported at the project level. What does this mean?
  * are there shared projects that can help resolve this discrepancy?
* How well do the EIA 860m and interconnection queues match?
  * Project counts
  * Capacity
  * Location
  * Fuel types
* How much lead time is there between projects appearing in each data source and project completion?
  * does this change by fuel type, location, etc?

## Load the data

In [52]:
engine = dbcp.helpers.get_sql_engine()

In [53]:
all_proj_long = dbcp.data_mart.projects.create_long_format(engine, active_projects_only=False)

In [54]:
all_proj_long.duplicated(subset=['source', 'project_id', 'resource_clean', 'county_id_fips']).sum()

0

In [55]:
# the change log is more complicated to work with because it has multiple entries for each operational/withdrawn project
# I'll just use the all_proj_long instead
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        pd.read_sql(
            "select source, project_id, resource_clean, county_id_fips, queue_status, effective_date, end_date, interconnection_status, capacity_mw from data_mart.iso_projects_change_log order by 1,2,3,4,5,6 limit 10",
            engine,
        )
    )

,source,project_id,resource_clean,county_id_fips,queue_status,effective_date,end_date,interconnection_status,capacity_mw
0,gridstatus,32512,Oil,42091,new,2011-12-30 00:00:00,2012-12-31 00:00:00,None,4.0
1,gridstatus,32512,Oil,42091,withdrawn,2012-12-31 00:00:00,NaT,None,4.0
2,gridstatus,32513,Onshore Wind,27081,new,2002-03-04 05:00:00,2007-10-01 04:00:00,Network Upgrade,100.0
3,gridstatus,32513,Onshore Wind,27081,operational,2007-10-01 04:00:00,NaT,Network Upgrade,100.0
4,gridstatus,32515,Onshore Wind,27081,new,2007-02-06 05:00:00,2008-11-14 05:00:00,SPA - SIS,300.0
5,gridstatus,32515,Onshore Wind,27081,withdrawn,2008-11-14 05:00:00,NaT,SPA - SIS,300.0
6,gridstatus,32516,Natural Gas,36121,new,2009-03-06 00:00:00,2009-04-08 00:00:00,Withdrawn,2.0
7,gridstatus,32516,Natural Gas,36121,withdrawn,2009-04-08 00:00:00,NaT,Withdrawn,2.0
8,gridstatus,32517,Solar,36101,new,2014-09-09 00:00:00,2015-07-31 00:00:00,Withdrawn,0.7
9,gridstatus,32517,Solar,36101,withdrawn,2015-07-31 00:00:00,NaT,Withdrawn,0.7


In [56]:
# copy a few nan-handling bits from the create_project_change_log code
all_proj_long['end_date'] = pd.Timestamp(np.nan)
# for all projects, the "start date" is date_entered_queue
is_operational = all_proj_long.loc[:,'queue_status'].eq('operational')
is_withdrawn = all_proj_long.loc[:,'queue_status'].eq('withdrawn')

# active projects have no end date and suspended projects have no information on when they became suspended.
# There are not many suspended projects, so I'll just ignore them for now
# is_active = all_proj_long.loc[:,'queue_status'].eq('active')
# is_suspended = all_proj_long.loc[:,'queue_status'].eq('suspended')

all_proj_long.loc[is_operational, 'end_date'] = all_proj_long['actual_completion_date'].fillna(all_proj_long['date_proposed_online']).loc[is_operational]
all_proj_long.loc[is_withdrawn, 'end_date'] = all_proj_long.loc[is_withdrawn, 'withdrawn_date']

In [57]:
all_proj_long.sample(5)

,state,county,queue_id,is_nearly_certain,project_id,project_name,capacity_mw,developer,entity,iso_region,utility,date_proposed_online,point_of_interconnection,is_actionable,resource_clean,queue_status,date_entered_queue,actual_completion_date,withdrawn_date,interconnection_status,state_id_fips,county_id_fips,frac_locations_in_county,source,state_permitting_type,co2e_tonnes_per_year,ordinance_earliest_year_mentioned,ordinance_jurisdiction_name,ordinance_jurisdiction_type,ordinance_text,ordinance_via_reldi,ordinance_via_solar_nrel,ordinance_via_wind_nrel,ordinance_via_nrel_is_de_facto,ordinance_via_self_maintained,ordinance_is_restrictive,is_hybrid,resource_class,surrogate_id,end_date
19235,Arizona,Pinal,Q382,None,95,None,75.00,None,APS,West (non-ISO),APS,2024-12-15 00:00:00,New substatio off the Santa Rosa - Vista\n69 k...,None,Solar,withdrawn,2021-03-31,NaT,NaT,Withdrawn,04,04021,1.0,lbnl,Hybrid,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False,False,True,renewable,19235,NaT
16925,Texas,Ector,26INR0279,False,49601,Firebird Solar,100.42,"TX Firebird SB 6, LLC",ERCOT,ERCOT,None,2026-12-31 00:00:00,138 kV 1099 SOUTHWESTERN PORTLAND TAP - 11281 ...,True,Solar,active,2023-11-14,NaT,NaT,"SS Completed, FIS Started, No IA",48,48135,1.0,gridstatus,Local,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,None,False,False,renewable,16925,NaT
4129,Rhode Island,Newport,805,False,36659,Solar,10.99,None,ISONE,ISONE,NGRID,2024-03-22 00:00:00,GRID Jepson 37K33,False,Solar,active,2018-12-10,2024-03-29,NaT,None,44,44005,1.0,gridstatus,Local,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,None,False,False,renewable,4129,NaT
18846,Kansas,Elk,GEN-2005-016,None,51527,None,150.00,None,SPP,SPP,WERE,NaT,None,None,Onshore Wind,withdrawn,2005-09-14,NaT,NaT,WITHDRAWN,20,20049,1.0,gridstatus,Local,NaN,NaN,NaN,NaN,NaN,False,False,True,False,None,True,False,renewable,18846,NaT
4606,Kansas,Thomas,GEN-2016-067,None,37139,None,73.60,None,SPP,SPP,SUNC,2019-07-24 00:00:00,Mingo 345kV,None,Onshore Wind,operational,2016-03-31,NaT,NaT,IA FULLY EXECUTED/COMMERCIAL OPERATION,20,20193,1.0,gridstatus,Local,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,None,False,False,renewable,4606,2019-07-24


In [58]:
# start with current data only. Can use full changelog if needed later
eia_current = pd.read_sql("select * from data_mart.projects_current_860m", engine)
eia_current.sample(5)

,report_date,plant_name_eia,plant_id_eia,generator_id,utility_id_eia,utility_name_eia,operational_status_code,operational_status_category,fuel_type_code_pudl,capacity_mw,state,county,prime_mover_code,current_planned_generator_operating_date,energy_source_code_1,energy_storage_capacity_mwh,generator_retirement_date,latitude,longitude,net_capacity_mwdc,raw_operational_status_code,planned_derate_date,planned_generator_retirement_date,planned_net_summer_capacity_derate_mw,planned_net_summer_capacity_uprate_mw,planned_uprate_date,technology_description,state_id_fips,county_id_fips
19026,2017-12-01,EG178 Facility,56233,CT02,49949,Kinder Morgan Production Company LP,7,existing,gas,60.5,TX,Scurry,CT,NaT,NG,NaN,NaT,32.747059,-100.954697,None,OP,NaT,NaT,NaN,NaN,NaT,Natural Gas Fired Combined Cycle,48,48415
14268,2019-02-01,Salmon Falls,50702,GEN2,7601,Green Mountain Power Corp,7,existing,hydro,0.4,ME,York,HY,NaT,WAT,NaN,NaT,43.226978,-70.810890,None,OP,NaT,NaT,NaN,NaN,NaT,Conventional Hydroelectric,23,23031
24572,2023-03-01,Tarboro Solar,59648,5MWPV,65411,Duke Energy Renewables Services,7,existing,solar,5.0,NC,Edgecombe,PV,NaT,SUN,NaN,NaT,35.858055,-77.499168,None,OP,NaT,NaT,NaN,NaN,NaT,Solar Photovoltaic,37,37065
9942,2017-03-01,Kings Beach,6518,2,57222,Liberty Utilities (CalPeco Electric) LLC,8,retired,oil,2.7,CA,Placer,IC,NaT,DFO,NaN,2008-03-01,39.245834,-120.027199,None,RE,NaT,NaT,NaN,NaN,NaT,Petroleum Liquids,06,06061
1937,2016-09-01,Puueo,771,1,8287,Hawaii Electric Light Co Inc,7,existing,hydro,0.7,HI,Hawaii,HY,NaT,WAT,NaN,NaT,19.726400,-155.090805,None,OP,NaT,NaT,NaN,NaN,NaT,Conventional Hydroelectric,15,15001


In [59]:
eia_dates = pd.read_sql("select * from data_mart.projects_transition_dates_860m", engine)

In [60]:
eia_dates.sample(5, random_state=42)

,plant_name_eia,plant_id_eia,generator_id,date_entered_1,date_entered_2,date_entered_3,date_entered_4,date_entered_5,date_entered_6,date_entered_7,date_entered_8,date_entered_99
13810,Vallecito Hydroelectric,50206,GEN2,NaT,NaT,NaT,NaT,NaT,NaT,2015-07-01,NaT,NaT
22107,Oakley Generating Station,57552,CT2,NaT,NaT,NaT,2015-07-01,NaT,NaT,NaT,NaT,NaT
27550,Anchor Energy,61304,GEN5,2017-06-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
15987,John Deere Dubuque Works,54414,GE10,NaT,NaT,NaT,NaT,NaT,NaT,2015-07-01,NaT,NaT
16171,Venice Resources Gas Recovery,54539,GEN1,NaT,NaT,NaT,NaT,NaT,NaT,2015-07-01,NaT,NaT


In [61]:
# need to join on ISO region to 860m data
# the following query refers to the pudl.sqlite database (v2024.2.6). I exported a csv to load it here.
query = """
WITH
bas as (
    -- Get the most recent entry for each balancing authority.
    -- This table includes every BA that has ever existed.
    SELECT
        balancing_authority_id_eia,
        balancing_authority_code_eia,
        balancing_authority_name_eia
    FROM (
        SELECT
            balancing_authority_id_eia,
            balancing_authority_code_eia,
            balancing_authority_name_eia,
            ROW_NUMBER() OVER (PARTITION BY balancing_authority_id_eia ORDER BY report_date DESC) AS report_order
        FROM core_eia861__yearly_balancing_authority
        ) as t
    WHERE report_order = 1
),
assoc as (
    -- The purpose of this CTE is to resolve the rare (1.5%) m:m relationships
    -- between utility-states and balancing authorities down to m:1 relationships.
    SELECT
        t.balancing_authority_id_eia,
        u.utility_name_eia,
        t.utility_id_eia,
        t.state,
        balancing_authority_code_eia,
        balancing_authority_name_eia,
        ROW_NUMBER() OVER (  -- Rank m:m relationships
            PARTITION BY t.utility_id_eia, t.state
            ORDER BY
                -- Prioritize ISOs over other BAs
                balancing_authority_code_eia in ('MISO', 'PJM', 'CISO', 'NYIS', 'SWPP', 'ERCO', 'ISNE') DESC,
                -- Where there are multiple ISOs, arbitrarily but consistently choose the one with the lowest ID
                t.balancing_authority_id_eia ASC
            ) as tiebreaker
    FROM core_eia861__assn_balancing_authority as t
    LEFT JOIN bas
    ON t.balancing_authority_id_eia = bas.balancing_authority_id_eia
    LEFT JOIN core_eia861__yearly_utility_data_misc as u
    ON t.utility_id_eia = u.utility_id_eia AND u.report_date = '2019-01-01'
    WHERE t.report_date = '2019-01-01'
)
SELECT
    utility_id_eia,
    state,
    utility_name_eia,
    balancing_authority_id_eia,
    balancing_authority_code_eia,
    balancing_authority_name_eia
FROM assoc
WHERE tiebreaker = 1
ORDER BY utility_id_eia, state
;
"""
eia_iso = pd.read_csv("./860m_isos.csv")
eia_iso.sample(3)

,report_date,utility_id_eia,state,utility_name_eia,balancing_authority_id_eia,balancing_authority_code_eia,balancing_authority_name_eia
3317,2005-01-01,1279,MO,NaN,924,AECI,"Associated Electric Cooperative, Inc."
29860,2003-01-01,11011,KY,NaN,5580,NaN,East Kentucky Power Cooperative
6411,2008-01-01,2304,IN,Town of Brookston,13756,NaN,Northern Indiana Pub Serv Co


In [64]:
eia_iso['report_date'] = pd.to_datetime(eia_iso['report_date'])

In [67]:
eia_current['report_date'].value_counts().head(5)

2017-03-01    6201
2016-07-01    5595
2022-04-01    4773
2023-02-01    1136
2023-01-01     892
Name: report_date, dtype: int64

In [68]:
eia_iso['report_date'].value_counts().head(5)

2019-01-01    4169
2020-01-01    3662
2022-01-01    3661
2021-01-01    3635
2018-01-01    3499
Name: report_date, dtype: int64

In [72]:
g_cur = eia_current.groupby(['utility_id_eia', 'state', 'report_date']).size().index
g_iso = eia_iso.groupby(['utility_id_eia', 'state', 'report_date']).size().index
g_cur.shape, g_iso.shape

((11037,), (73371,))

In [73]:
g_cur.difference(g_iso).shape, g_iso.difference(g_cur).shape

((10875,), (73209,))

In [75]:
set(eia_current['state'].unique()).symmetric_difference(set(eia_iso['state'].unique()))

{'CN', 'UNK', 'VI'}

In [78]:
len(set(eia_current['utility_id_eia'].unique()).difference(set(eia_iso['utility_id_eia'].unique())))

5817

In [79]:
eia_current['utility_id_eia'].nunique(), eia_iso['utility_id_eia'].nunique()

(6616, 3636)

In [81]:
len(set(eia_current['report_date'].unique()).difference(set(eia_iso['report_date'].unique())))

84

In [82]:
eia_current['report_date'].agg(['min', 'max']), eia_iso['report_date'].agg(['min', 'max'])

(min   2016-04-01
 max   2023-11-01
 Name: report_date, dtype: datetime64[ns],
 min   2001-01-01
 max   2022-01-01
 Name: report_date, dtype: datetime64[ns])

In [80]:
eia_iso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73371 entries, 0 to 73370
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   report_date                   73371 non-null  datetime64[ns]
 1   utility_id_eia                73371 non-null  int64         
 2   state                         73371 non-null  object        
 3   utility_name_eia              47621 non-null  object        
 4   balancing_authority_id_eia    73371 non-null  int64         
 5   balancing_authority_code_eia  60895 non-null  object        
 6   balancing_authority_name_eia  73371 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 3.9+ MB


In [65]:
eia_current = eia_current.merge(eia_iso.drop(columns=['utility_name_eia']), how='left', on=['utility_id_eia', 'state', 'report_date'])  # drop shared name column
eia_current.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33617 entries, 0 to 33616
Data columns (total 32 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   report_date                               33617 non-null  datetime64[ns]
 1   plant_name_eia                            33617 non-null  object        
 2   plant_id_eia                              33617 non-null  int64         
 3   generator_id                              33617 non-null  object        
 4   utility_id_eia                            33617 non-null  int64         
 5   utility_name_eia                          33617 non-null  object        
 6   operational_status_code                   33617 non-null  int64         
 7   operational_status_category               33617 non-null  object        
 8   fuel_type_code_pudl                       33614 non-null  object        
 9   capacity_mw                 

## Generator vs Project Level Data

Find some shared projects to compare between the EIA 860m and interconnection queues. Specifically, look for 860m generators at the same plant that have different queue statuses.

In [11]:
# how many plants have multiple status codes?
eia_current.groupby('plant_id_eia')['operational_status_code'].nunique().value_counts(dropna=False)

1    13995
2      864
3       19
5        1
Name: operational_status_code, dtype: int64

In [12]:
# filter down to only those with multiple status codes
multi_status = eia_current.groupby('plant_id_eia').filter(lambda df: df['operational_status_code'].nunique() > 1)
multi_status['operational_status_code'].value_counts(dropna=False)

7    3328
8    2147
1      38
5      36
4      30
6      15
2      15
3       9
Name: operational_status_code, dtype: int64

In [13]:
# what status combinations are there? [update: 802/884 (90.7%) are combinations of operating and retired]
multi_status.groupby('plant_id_eia')['operational_status_code'].unique().apply(set).value_counts(dropna=False)

{8, 7}             802
{1, 7}              15
{5, 7}               9
{8, 1, 7}            7
{6, 7}               7
{8, 5, 7}            6
{4, 7}               6
{8, 4, 7}            4
{4, 5}               4
{1, 2}               3
{1, 4}               3
{3, 7}               2
{5, 6}               2
{8, 4}               2
{2, 7}               2
{8, 6, 7}            2
{8, 5}               2
{1, 5}               1
{2, 3, 4, 5, 7}      1
{2, 3}               1
{8, 6}               1
{8, 3}               1
{3, 6}               1
Name: operational_status_code, dtype: int64

In [14]:
# spot check a few
status_combos = multi_status.groupby('plant_id_eia')['operational_status_code'].unique().apply(set).rename('status_combo').to_frame()
def check_common_combos(s: set) -> str:  # mapping
    if s == {8,7}:
        return 'ret_op'
    elif 7 in s or 8 in s:
        return 'has_existing'
    else:
        return 'other'
status_combos['simplified'] = status_combos['status_combo'].apply(check_common_combos)
status_combos['simplified'].value_counts(dropna=False)

ret_op          802
has_existing     67
other            15
Name: simplified, dtype: int64

In [24]:
sample_ret_op = status_combos.query("simplified == 'ret_op'").sample(5, random_state=42).index
sample_has_existing = status_combos.query("simplified == 'has_existing'").sample(5, random_state=42).index
sample_other = status_combos.query("simplified == 'other'").index #.sample(5, random_state=42).index
eia_current.loc[eia_current['plant_id_eia'].isin(sample_ret_op), :].sort_values(['plant_id_eia', 'generator_id']).drop(columns=['utility_name_eia', 'technology_description']) # drop 2 long cols that make the df print too big

,report_date,plant_name_eia,plant_id_eia,generator_id,utility_id_eia,operational_status_code,operational_status_category,fuel_type_code_pudl,capacity_mw,state,county,prime_mover_code,current_planned_generator_operating_date,energy_source_code_1,energy_storage_capacity_mwh,generator_retirement_date,latitude,longitude,net_capacity_mwdc,raw_operational_status_code,planned_derate_date,planned_generator_retirement_date,planned_net_summer_capacity_derate_mw,planned_net_summer_capacity_uprate_mw,planned_uprate_date,state_id_fips,county_id_fips
3417,2019-02-01,E W Brown,1355,1,10171,8,retired,coal,113.599998,KY,Mercer,ST,NaT,BIT,NaN,2019-02-01,37.788311,-84.712570,None,RE,NaT,NaT,NaN,NaN,NaT,21,21167
3418,2016-07-01,E W Brown,1355,10,10171,7,existing,gas,126.000000,KY,Mercer,GT,NaT,NG,NaN,NaT,37.788311,-84.712570,None,OP,NaT,NaT,NaN,NaN,NaT,21,21167
3419,2016-07-01,E W Brown,1355,11,10171,7,existing,gas,126.000000,KY,Mercer,GT,NaT,NG,NaN,NaT,37.788311,-84.712570,None,OP,NaT,NaT,NaN,NaN,NaT,21,21167
3420,2019-02-01,E W Brown,1355,2,10171,8,retired,coal,179.500000,KY,Mercer,ST,NaT,BIT,NaN,2019-02-01,37.788311,-84.712570,None,RE,NaT,NaT,NaN,NaN,NaT,21,21167
3421,2020-02-01,E W Brown,1355,3,10171,7,existing,coal,464.000000,KY,Mercer,ST,NaT,BIT,NaN,NaT,37.788311,-84.712570,None,OP,NaT,NaT,NaN,NaN,NaT,21,21167
3422,2016-07-01,E W Brown,1355,5,10171,7,existing,gas,123.000000,KY,Mercer,GT,NaT,NG,NaN,NaT,37.788311,-84.712570,None,OP,NaT,NaT,NaN,NaN,NaT,21,21167
3423,2016-07-01,E W Brown,1355,6,10171,7,existing,gas,176.800003,KY,Mercer,GT,NaT,NG,NaN,NaT,37.788311,-84.712570,None,OP,NaT,NaT,NaN,NaN,NaT,21,21167
3424,2016-07-01,E W Brown,1355,7,10171,7,existing,gas,176.800003,KY,Mercer,GT,NaT,NG,NaN,NaT,37.788311,-84.712570,None,OP,NaT,NaT,NaN,NaN,NaT,21,21167
3425,2016-07-01,E W Brown,1355,8,10171,7,existing,gas,126.000000,KY,Mercer,GT,NaT,NG,NaN,NaT,37.788311,-84.712570,None,OP,NaT,NaT,NaN,NaN,NaT,21,21167
3426,2016-07-01,E W Brown,1355,9,10171,7,existing,gas,126.000000,KY,Mercer,GT,NaT,NG,NaN,NaT,37.788311,-84.712570,None,OP,NaT,NaT,NaN,NaN,NaT,21,21167


In [16]:
eia_current.loc[eia_current['plant_id_eia'].isin(sample_has_existing), :].sort_values(['plant_id_eia', 'generator_id']).drop(columns=['utility_name_eia', 'technology_description'])

,report_date,plant_name_eia,plant_id_eia,generator_id,utility_id_eia,operational_status_code,operational_status_category,fuel_type_code_pudl,capacity_mw,state,county,prime_mover_code,current_planned_generator_operating_date,energy_source_code_1,energy_storage_capacity_mwh,generator_retirement_date,latitude,longitude,net_capacity_mwdc,raw_operational_status_code,planned_derate_date,planned_generator_retirement_date,planned_net_summer_capacity_derate_mw,planned_net_summer_capacity_uprate_mw,planned_uprate_date,state_id_fips,county_id_fips
3120,2023-07-01,Colby City of,1272,10,3913,5,proposed,oil,3.000000,KS,Thomas,IC,2023-12-01,DFO,NaN,NaT,39.394413,-101.058403,None,V,NaT,NaT,NaN,NaN,NaT,20,20193
3121,2018-12-01,Colby City of,1272,3,3913,8,retired,oil,2.500000,KS,Thomas,IC,NaT,DFO,NaN,2018-04-01,39.394413,-101.058403,None,RE,NaT,NaT,NaN,NaN,NaT,20,20193
3122,2017-03-01,Colby City of,1272,4,3913,7,existing,oil,1.700000,KS,Thomas,IC,NaT,DFO,NaN,NaT,39.394413,-101.058403,None,OP,NaT,NaT,NaN,NaN,NaT,20,20193
3123,2017-03-01,Colby City of,1272,5,3913,7,existing,oil,1.300000,KS,Thomas,IC,NaT,DFO,NaN,NaT,39.394413,-101.058403,None,OP,NaT,NaT,NaN,NaN,NaT,20,20193
3124,2021-12-01,Colby City of,1272,6,3913,7,existing,oil,4.500000,KS,Thomas,IC,NaT,DFO,NaN,NaT,39.394413,-101.058403,None,OA,NaT,NaT,NaN,NaN,NaT,20,20193
3125,2017-03-01,Colby City of,1272,7,3913,7,existing,oil,4.500000,KS,Thomas,IC,NaT,DFO,NaN,NaT,39.394413,-101.058403,None,OP,NaT,NaT,NaN,NaN,NaT,20,20193
3126,2017-03-01,Colby City of,1272,8,3913,7,existing,oil,2.700000,KS,Thomas,IC,NaT,DFO,NaN,NaT,39.394413,-101.058403,None,OP,NaT,NaT,NaN,NaN,NaT,20,20193
3127,2019-08-01,Colby City of,1272,9,3913,7,existing,oil,3.000000,KS,Thomas,IC,NaT,DFO,NaN,NaT,39.394413,-101.058403,None,OP,NaT,NaT,NaN,NaN,NaT,20,20193
6204,2016-07-01,Plant No 1 Freeport,2678,1,6775,8,retired,oil,2.100000,NY,Nassau,IC,NaT,DFO,NaN,2013-06-01,40.656101,-73.592201,None,RE,NaT,NaT,NaN,NaN,NaT,36,36059
6205,2016-07-01,Plant No 1 Freeport,2678,2,6775,7,existing,oil,2.900000,NY,Nassau,IC,NaT,DFO,NaN,NaT,40.656101,-73.592201,None,OP,NaT,NaT,NaN,NaN,NaT,36,36059


In [25]:
# mostly solar + battery projects where the battery is in an earlier stage of development
eia_current.loc[eia_current['plant_id_eia'].isin(sample_other), :].sort_values(['plant_id_eia', 'generator_id']).drop(columns=['utility_name_eia', 'technology_description'])

,report_date,plant_name_eia,plant_id_eia,generator_id,utility_id_eia,operational_status_code,operational_status_category,fuel_type_code_pudl,capacity_mw,state,county,prime_mover_code,current_planned_generator_operating_date,energy_source_code_1,energy_storage_capacity_mwh,generator_retirement_date,latitude,longitude,net_capacity_mwdc,raw_operational_status_code,planned_derate_date,planned_generator_retirement_date,planned_net_summer_capacity_derate_mw,planned_net_summer_capacity_uprate_mw,planned_uprate_date,state_id_fips,county_id_fips
30218,2023-11-01,Copperton Solar Plant No. 1,64427,CSP1,49805,5,proposed,solar,5.000000,UT,Salt Lake,PV,2024-02-01,SUN,NaN,NaT,40.576641,-112.084602,None,V,NaT,NaT,NaN,NaN,NaT,49,49035
30219,2023-01-01,Copperton Solar Plant No. 1,64427,CSP2,49805,1,proposed,solar,25.000000,UT,Salt Lake,PV,2025-10-01,SUN,NaN,NaT,40.576641,-112.084602,None,P,NaT,NaT,NaN,NaN,NaT,49,49035
30464,2022-12-01,Rexford Solar Farm,64633,20SD8,64247,1,proposed,solar,300.000000,CA,Tulare,PV,2024-12-01,SUN,NaN,NaT,35.840481,-119.082703,None,P,NaT,NaT,NaN,NaN,NaT,06,06107
30465,2023-11-01,Rexford Solar Farm,64633,20SDB,64247,4,proposed,other,300.000000,CA,Tulare,BA,2024-12-01,MWH,NaN,NaT,35.840481,-119.082703,None,U,NaT,NaT,NaN,NaN,NaT,06,06107
31558,2023-07-01,Rose Gold Solar,65471,RGS01,50123,3,proposed,solar,150.000000,IN,Jay,PV,2025-12-01,SUN,NaN,NaT,40.404358,-85.197159,None,T,NaT,NaT,NaN,NaN,NaT,18,18075
31559,2023-07-01,Rose Gold Solar,65471,RGS23,50123,2,proposed,solar,100.000000,IN,Jay,PV,2027-12-01,SUN,NaN,NaT,40.404358,-85.197159,None,L,NaT,NaT,NaN,NaN,NaT,18,18075
32036,2023-10-01,Vikings Energy Farm,65711,GEN1,64966,4,proposed,solar,136.800003,CA,Imperial,PV,2024-05-01,SUN,NaN,NaT,32.803364,-115.271698,None,U,NaT,NaT,NaN,NaN,NaT,06,06025
32037,2023-08-01,Vikings Energy Farm,65711,GEN2,64966,5,proposed,other,150.000000,CA,Imperial,BA,2024-02-01,MWH,NaN,NaT,32.803364,-115.271698,None,V,NaT,NaT,NaN,NaN,NaT,06,06025
32239,2023-04-01,"Springwater Solar, LLC",65900,SPRI2,65092,1,proposed,other,75.000000,OH,Madison,BA,2026-02-01,MWH,NaN,NaT,39.829720,-83.249802,None,P,NaT,NaT,NaN,NaN,NaT,39,39097
32240,2022-08-01,"Springwater Solar, LLC",65900,SPRIN,65092,2,proposed,solar,155.000000,OH,Madison,PV,2024-11-01,SUN,NaN,NaT,39.829720,-83.249802,None,L,NaT,NaT,NaN,NaN,NaT,39,39097


### Look for shared projects

One takeaway was that the ISO project to EIA generator relationships are not consistent. Often there are multiple projects that could potentially match with a proposed EIA generator. Hybrid projects are split into two entries in some ISOs (PJM)and remain a single project in others (CAISO).

In [17]:
# ISO queues have many more active proposals than completed projects. EIA data is the opposite.
queue_shared_cols = ['utility', 'state', 'county', 'county_id_fips', 'capacity_mw', 'resource_clean', 'queue_status', 'interconnection_status', 'date_entered_queue', 'end_date']
queue_idx_cols = ['project_id', 'queue_id', 'source']
eia_shared_cols = ['state', 'county', 'county_id_fips', 'capacity_mw', 'fuel_type_code_pudl', 'operational_status_category', 'operational_status_code', 'report_date']
eia_idx_cols = ['plant_id_eia', 'generator_id']

op_q = all_proj_long.loc[all_proj_long['queue_status'].eq('operational'), queue_idx_cols + queue_shared_cols].set_index('project_id').sort_index()
active_q = all_proj_long.loc[all_proj_long['queue_status'].eq('active'), queue_idx_cols + queue_shared_cols].set_index('project_id').sort_index()
op_eia = eia_current.loc[eia_current['operational_status_category'].eq('existing'), eia_idx_cols + eia_shared_cols].set_index(eia_idx_cols).sort_index()
active_eia = eia_current.loc[eia_current['operational_status_category'].eq('proposed'), eia_idx_cols + eia_shared_cols].set_index(eia_idx_cols).sort_index()
len(op_q), len(active_q), len(op_eia), len(active_eia)

(3725, 12541, 25748, 1581)

In [18]:
active_eia.groupby(level='plant_id_eia', as_index=False)['operational_status_code'].nunique().value_counts(dropna=False)

operational_status_code
1                          1228
2                            15
4                             1
dtype: int64

In [19]:
active_eia['operational_status_code'].value_counts(dropna=False)

1     416
5     349
4     294
2     226
3     161
6     134
99      1
Name: operational_status_code, dtype: int64

In [20]:
# look at some recently completed queue projects
op_q.query("date_entered_queue > '2020-01-01'").sample(10, random_state=42)

,queue_id,source,utility,state,county,county_id_fips,capacity_mw,resource_clean,queue_status,interconnection_status,date_entered_queue,end_date
project_id,,,,,,,,,,,,
31250,83,lbnl,TEC,Florida,Hillsborough,12057,0.00,Natural Gas,operational,Operational,2021-12-13,2023-04-01
31678,419,lbnl,None,Tennessee,Rhea,47143,0.00,Nuclear,operational,Operational,2020-02-28,NaT
47224,AF2-250,gridstatus,DPL,Maryland,Wicomico,24045,1.95,Solar,operational,None,2020-03-27,2022-10-19
8666,GI-086,lbnl,GTC,Georgia,Calhoun,13037,80.00,Solar,operational,Operational,2023-12-31,NaT
9333,596,lbnl,IP,Idaho,Twin Falls,16083,2.50,Hydro,operational,Operational,2020-06-30,2021-05-01
8406,378B,lbnl,FPL,Florida,Jackson,12063,75.00,Solar,operational,Operational,2020-02-04,2022-10-01
15945,398,lbnl,NWMT,Montana,Cascade,30013,2.00,Hydro,operational,Operational,2020-11-16,NaT
8660,GI-020,lbnl,GTC,Georgia,Lee,13177,150.00,Natural Gas,operational,Operational,2023-12-31,NaT
9524,579,lbnl,IP,Idaho,Twin Falls,16083,2.60,Hydro,operational,Operational,2020-04-23,2020-05-01


In [21]:
op_eia.query("county_id_fips == '12063'")

state   county county_id_fips  capacity_mw fuel_type_code_pudl operational_status_category  operational_status_code report_date
plant_id_eia generator_id                                                                                                                                
56522        GEN1            FL  Jackson          12063          0.8               waste                    existing                        7  2017-03-01
             GEN2            FL  Jackson          12063          0.8               waste                    existing                        7  2017-03-01
             GEN3            FL  Jackson          12063          0.8               waste                    existing                        7  2017-03-01
             GEN4            FL  Jackson          12063          0.8               waste                    existing                        7  2017-03-01
             GEN5            FL  Jackson          12063          0.8               waste                    existing                        7  2017-03-01
             GEN6            FL  Jackson          12063          0.8               waste                    existing                        7  2017-03-01
63754        SBI01           FL  Jackson          12063         74.5               solar                    existing                        7  2022-10-01
64757        1               FL  Jackson          12063         74.5               solar                    existing                        7  2022-04-01
65432        1               FL  Jackson          12063         74.5               solar                    existing                        7  2023-01-01

In [22]:
eia_current.loc[eia_current['plant_id_eia'].isin({63754, 64757, 65432}),:]

,report_date,plant_name_eia,plant_id_eia,generator_id,utility_id_eia,utility_name_eia,operational_status_code,operational_status_category,fuel_type_code_pudl,capacity_mw,state,county,prime_mover_code,current_planned_generator_operating_date,energy_source_code_1,energy_storage_capacity_mwh,generator_retirement_date,latitude,longitude,net_capacity_mwdc,raw_operational_status_code,planned_derate_date,planned_generator_retirement_date,planned_net_summer_capacity_derate_mw,planned_net_summer_capacity_uprate_mw,planned_uprate_date,technology_description,state_id_fips,county_id_fips
29491,2022-10-01,Gulf Power Blue Indigo Energy,63754,SBI01,6452,Florida Power & Light Co,7,existing,solar,74.5,FL,Jackson,PV,NaT,SUN,NaN,NaT,30.877262,-85.400650,None,OP,NaT,NaT,NaN,NaN,NaT,Solar Photovoltaic,12,12063
30587,2022-04-01,Blue Springs,64757,1,6452,Florida Power & Light Co,7,existing,solar,74.5,FL,Jackson,PV,NaT,SUN,NaN,NaT,30.782898,-85.078690,None,OP,NaT,NaT,NaN,NaN,NaT,Solar Photovoltaic,12,12063
31513,2023-01-01,Apalachee,65432,1,6452,Florida Power & Light Co,7,existing,solar,74.5,FL,Jackson,PV,NaT,SUN,NaN,NaT,30.760550,-85.069519,None,OP,NaT,NaT,NaN,NaN,NaT,Solar Photovoltaic,12,12063


In [27]:
# look at some proposed EIA projects
active_eia.query("report_date > '2020-01-01'").sample(10, random_state=42)

,,state,county,county_id_fips,capacity_mw,fuel_type_code_pudl,operational_status_category,operational_status_code,report_date
plant_id_eia,generator_id,,,,,,,,
66435,USCOS,MN,Renville,27129,1.000000,solar,proposed,6,2023-11-01
65853,MIDPV,TX,Nacogdoches,48347,609.099976,solar,proposed,2,2023-11-01
66529,21013,NY,St Lawrence,None,5.200000,solar,proposed,6,2023-11-01
64850,WS,NC,Wilkes,37193,75.000000,solar,proposed,1,2023-05-01
64669,KOV4A,UT,Millard,49027,324.000000,solar,proposed,3,2021-04-01
66993,717,WI,Dunn,55033,1.500000,solar,proposed,4,2023-11-01
64745,BHS01,CO,Weld,08123,150.000000,solar,proposed,1,2023-07-01
62523,1093,NY,Ontario,36069,2.300000,solar,proposed,2,2023-08-01
64551,WT2,TX,Callahan,48059,114.900002,wind,proposed,6,2023-11-01


In [32]:
# easy match for the big solar plant
active_q.query("county_id_fips == '48347' and resource_clean == 'Solar'")

,queue_id,source,utility,state,county,county_id_fips,capacity_mw,resource_clean,queue_status,interconnection_status,date_entered_queue,end_date
project_id,,,,,,,,,,,,
39073,24INR0418,gridstatus,None,Texas,Nacogdoches,48347,609.10,Solar,active,"SS Completed, FIS Started, No IA",2022-10-18,NaT
40535,26INR0161,gridstatus,None,Texas,Nacogdoches,48347,201.00,Solar,active,"SS Completed, FIS Started, No IA",2023-08-14,NaT
46336,27INR0226,gridstatus,None,Texas,Nacogdoches,48347,206.71,Solar,active,"SS Started, FIS Started, No IA",2024-03-29,NaT


In [ ]:
# no clear match for the wind plant
active_q.query("county_id_fips == '48059' and resource_clean == 'Onshore Wind'")

,queue_id,source,utility,state,county,county_id_fips,capacity_mw,resource_clean,queue_status,interconnection_status,date_entered_queue,end_date
project_id,,,,,,,,,,,,
39210,21INR0325,gridstatus,None,Texas,Callahan,48059,153.0,Onshore Wind,active,"SS Completed, FIS Completed, IA",2019-05-30,NaT
39225,20INR0083,gridstatus,None,Texas,Callahan,48059,350.0,Onshore Wind,active,"SS Completed, FIS Completed, IA",2018-05-04,NaT


In [34]:
# easy match for the 324MW solar plant
active_q.query("county_id_fips == '49027' and resource_clean == 'Solar'")

,queue_id,source,utility,state,county,county_id_fips,capacity_mw,resource_clean,queue_status,interconnection_status,date_entered_queue,end_date
project_id,,,,,,,,,,,,
10819,Q90,lbnl,LADWP,Utah,Millard,49027,400.0,Solar,active,Not Started,2022-04-01,NaT
10822,Q93,lbnl,LADWP,Utah,Millard,49027,1000.0,Solar,active,Not Started,2022-08-02,NaT
10825,Q49,lbnl,LADWP,Utah,Millard,49027,300.0,Solar,active,In Progress (unknown study),2015-08-25,NaT
10830,Q50,lbnl,LADWP,Utah,Millard,49027,400.0,Solar,active,In Progress (unknown study),2016-01-05,NaT
10840,Q53,lbnl,LADWP,Utah,Millard,49027,285.0,Solar,active,In Progress (unknown study),2016-07-28,NaT
10845,Q52,lbnl,LADWP,Utah,Millard,49027,324.0,Solar,active,In Progress (unknown study),2016-01-14,NaT
27062,C3-125,lbnl,PacifCorp,Utah,Millard,49027,75.0,Solar,active,Cluster Study,2023-05-15,NaT
27324,C3-101,lbnl,PacifCorp,Utah,Millard,49027,300.0,Solar,active,Cluster Study,2023-05-12,NaT
27970,C3-110,lbnl,PacifCorp,Utah,Millard,49027,250.0,Solar,active,Cluster Study,2023-05-15,NaT


## Compare Lead Times

For each data source, how much time passes between first reporting a project and the project being completed? This analysis is complicated by the varying availability of `actual_completion_date`.

In [38]:
# it is all over the place and nonsensical. How can ISONE have actual completion dates for active projects??
all_proj_long.groupby(['queue_status', 'iso_region'])['actual_completion_date'].agg(['count', 'size'])

count  size
queue_status iso_region                      
active       CAISO                    8  1410
             ERCOT                   99  1494
             ISONE                  480   481
             MISO                     0  1647
             NYISO                    0   449
             PJM                      1  2797
             SPP                      0   628
             Southeast (non-ISO)      8  1072
             West (non-ISO)           0  2542
operational  CAISO                  190   197
             ISONE                  309   312
             MISO                     0   475
             NYISO                   91   105
             PJM                   1098  1134
             SPP                      0   258
             Southeast (non-ISO)    142   332
             West (non-ISO)          44   912
suspended    PJM                      0   139
             SPP                      0     5
             Southeast (non-ISO)      0    44
             West (non-ISO)           0   213
withdrawn    CAISO                    0  1818
             ISONE                  826   834
             MISO                     0  2142
             NYISO                    0   824
             PJM                     63  3981
             SPP                      0  1437
             Southeast (non-ISO)     35  1823
             West (non-ISO)           0  4331

In [39]:
# are these processing errors? Wtf does this mean?
all_proj_long.query("queue_status == 'active' and iso_region == 'ISONE'").sample(10, random_state=42)

,state,county,queue_id,is_nearly_certain,project_id,project_name,capacity_mw,developer,entity,iso_region,utility,date_proposed_online,point_of_interconnection,is_actionable,resource_clean,queue_status,date_entered_queue,actual_completion_date,withdrawn_date,interconnection_status,state_id_fips,county_id_fips,frac_locations_in_county,source,state_permitting_type,co2e_tonnes_per_year,ordinance_earliest_year_mentioned,ordinance_jurisdiction_name,ordinance_jurisdiction_type,ordinance_text,ordinance_via_reldi,ordinance_via_solar_nrel,ordinance_via_wind_nrel,ordinance_via_nrel_is_de_facto,ordinance_via_self_maintained,ordinance_is_restrictive,is_hybrid,resource_class,surrogate_id,end_date
2070,Massachusetts,Barnstable,1283,False,34588,Mashpee 946 - EMA ASO,1.490,None,ISONE,ISONE,None,2023-09-30 00:00:00,Eversource Mashpee 946,False,Solar,active,2022-06-08,2023-11-30,NaT,None,25,25001,1.0,gridstatus,Local,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,None,False,False,renewable,2070,NaT
17816,Massachusetts,Worcester,1415,False,50492,Battery Storage,270.400,None,ISONE,ISONE,None,2024-07-12 00:00:00,"POI interconnection coordinates: 42.471953, -7...",False,Battery Storage,active,2023-06-20,2024-08-12,NaT,None,25,25027,1.0,gridstatus,Local,NaN,2018.0,Charlton,city,"In 2019, residents of the Town of Charlton vot...",True,NaN,NaN,NaN,None,True,False,storage,17816,NaT
17432,Massachusetts,Plymouth,1108,False,50108,Battery Storage,188.960,None,ISONE,ISONE,None,2027-02-12 00:00:00,"Eversource (NSTAR - South) 115 kV Line 116, 0....",False,Battery Storage,active,2021-03-15,2027-05-14,NaT,None,25,25023,1.0,gridstatus,Local,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,None,False,False,storage,17432,NaT
16134,Maine,Kennebec,1466,False,48809,Winslow-County Rd Lakewood 115 kV SS - Winslow...,22.960,None,ISONE,ISONE,None,2024-12-31 00:00:00,CMP Winslow-County Rd Lakewood 115 kV substation,False,Solar,active,2023-12-05,2024-12-31,NaT,None,23,23011,0.5,gridstatus,Hybrid,NaN,2020.0,multiple,city,"Albion: In March 2023, residents of the town o...",True,NaN,NaN,NaN,False,False,False,renewable,16134,NaT
17608,Connecticut,Hartford,939,True,50284,NB Fuel Cell,20.240,None,ISONE,ISONE,ISO-NE,2024-05-28 00:00:00,Black Rock Substation (Eversource) via the Far...,False,Natural Gas,active,2019-11-25,2024-06-15,NaT,None,09,09003,1.0,gridstatus,State,63356.501359,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,None,False,False,fossil,17608,NaT
7963,Maine,Aroostook,1227,False,40541,1200 MW non-controllable ETU - N to S flow only,1200.000,None,ISONE,ISONE,None,2026-10-08 00:00:00,New substation on Line 3023 near the Detroit S...,False,Unknown,active,2022-02-15,2026-12-09,NaT,None,23,23003,0.5,gridstatus,Hybrid,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,None,False,False,NaN,7963,NaT
17489,Massachusetts,Bristol,1477,False,50165,NG DG @ Sykes Road,9.998,None,ISONE,ISONE,None,2025-12-31 00:00:00,NG Sykes Rd 282TR (M13N),False,Battery Storage,active,2023-12-19,2025-12-31,NaT,None,25,25005,1.0,gridstatus,Local,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,None,False,False,storage,17489,NaT
17281,Connecticut,New Haven,1392,False,49957,Battery Storage,53.740,None,ISONE,ISONE,None,2026-08-15 00:00:00,"Latitude: 41.492036°, Longitude: -72.762080° -...",False,Battery Storage,active,2023-05-03,2026-09-15,NaT,None,09,09009,1.0,gridstatus,State,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,None,False,False,storage,17281,NaT
2074,Massachusetts,Barnstable,1004,False,34592,Eversource Group @ Sandwich 916,11.750,None,ISONE,ISONE,None,2023-09-30 00:00:00,Eversource Sandwich 916,False,Solar,active,2020-04-09,2023-11-30,NaT,None,25,25001,1.0,gridstatus,Local,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,None,False,False,renewable,2074,NaT
18204,New Hampshire,Rockingham,1461,False,50881,Battery Addition,1.498,None,ISONE,ISONE,None,2025-08-01 00:00:00,"400 Gosling Road, Portsmouth, New Hampshire. T...",False,Oil,active,2023-10-18,2025-09-01,NaT,None,33,33015,1.0,gridstatus,Hybrid,1003.641566,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,None,False,False,fossil

In [40]:
all_proj_long['duration_years'] = (all_proj_long['end_date'] - all_proj_long['date_entered_queue']).dt.days / 365.25

In [46]:
all_proj_long.query(
    "queue_status == 'operational' and actual_completion_date.notna() and end_date > '2015-01-01'"
).groupby(["resource_clean", "iso_region"])["duration_years"].agg(['count', 'min', 'median', 'mean', 'max']).query("count > 10")

count       min    median      mean        max
resource_clean  iso_region                                                         
Battery Storage CAISO                   16  2.149213  6.776181  7.103012  12.887064
                PJM                     24  0.802190  2.209446  3.236938   8.459959
Hydro           PJM                     13  0.848734  1.587953  2.748592   8.303901
Natural Gas     CAISO                   12  0.651608  3.990418  5.062058  12.197125
                ISONE                   36  0.457221  2.175222  2.772378   6.392882
                PJM                    236 -0.832307  3.030801  3.315352  10.798084
                Southeast (non-ISO)     21  0.186174  2.672142  2.483752   5.199179
Onshore Wind    ISONE                   30 -1.363450  4.158795  3.989231   8.544832
                NYISO                   11  4.188912  7.394935  7.279945  10.083504
                PJM                     44  1.095140  5.586585  6.189409  14.527036
Solar           CAISO                   63  1.267625  6.116359  6.763665  13.289528
                ISONE                   60 -3.425051  2.179329  2.381520   6.214921
                NYISO                   11  3.556468  4.605065  4.838778   6.970568
                PJM                    242  0.473648  4.179329  4.079318  11.080082
                Southeast (non-ISO)     47  1.130732  3.657769  3.599575   5.596167
                West (non-ISO)          24  1.248460  4.420260  4.608145  10.444901
Unknown         ISONE                   25  0.000000  2.108145  2.818234   9.089665

In [47]:
# same but using the estimated completion date due to missing the real one.
all_proj_long.query(
    "queue_status == 'operational' and actual_completion_date.isna() and end_date > '2015-01-01'"
).groupby(["resource_clean", "iso_region"])["duration_years"].agg(['count', 'min', 'median', 'mean', 'max']).query("count > 10")

count       min    median      mean        max
resource_clean  iso_region                                                         
Battery Storage West (non-ISO)          43  0.158795  2.680356  3.181383  11.972621
Hydro           West (non-ISO)          63 -0.057495  0.547570  0.936475   5.338809
Natural Gas     MISO                    35  0.000000  2.475017  2.805671  15.638604
                SPP                     11  1.516769  4.052019  3.548504   6.557153
                West (non-ISO)          19 -0.030116  1.366188  2.509312   7.044490
Onshore Wind    MISO                   100 -0.167009  2.809035  3.192142  11.397673
                SPP                    113  0.731006  4.665298  4.846892  16.073922
                West (non-ISO)          38  0.449008  2.435318  3.894665  12.804928
Solar           MISO                    37  0.221766  3.173169  2.886953   5.462012
                PJM                     23  1.563313  6.171116  6.017558   8.145106
                Southeast (non-ISO)     49  0.290212  2.272416  2.671975   6.516085
                West (non-ISO)         134  0.049281  2.364134  2.928479  15.553730
Unknown         West (non-ISO)          40  0.427105  4.588638  4.980014  13.412731